# VI deo
#### Rad sa velikim količinama podataka (Big Data) u pandasu može postati izazovan zbog ograničenja memorije i performansi. Pandas je optimizovan za rad s podacima koji mogu stati u memoriju, ali za veće datasetove neophodno je koristiti jednu od strategija za skaliranje.


In [2]:
import pandas as pd
df = pd.read_csv('../data/crimes.csv')

#### Učitali smo dataset koji ima čak milion redova i 28 kolona, dakle ukupno 28 miliona zapisa. 

In [3]:
df.shape

(1003448, 28)

## Strategije za skaliranje
#### Smanjivanje veličine dataseta korišćenjem tipova podataka koji zauzimaju manje memorije (npr. `float32` umesto `float64`, `int8` umesto `int64`).

In [4]:
df.dtypes

DR_NO               int64
Date Rptd          object
DATE OCC           object
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float64
LON               float64
dtype: object

#### Konvertovaćemo kolonu `DR_NO` iz int64 u int8, kao i `LAT` iz float64 u float32. Na kraju ćemo opet ispisati tipove kolona.

In [5]:
df['DR_NO'] = df['DR_NO'].astype('int8')
df['LAT'] = df['LAT'].astype('float32')
df.dtypes

DR_NO                int8
Date Rptd          object
DATE OCC           object
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float32
LON               float64
dtype: object

#### Još jedna mogućnost je učitavanje podskupa kolona pomoću parametra `usecols` pri učitavanju podataka. U našem slučaju, umesto da radimo sa 28 kolona (28 miliona podataka), možemo izdvojiti 4 ključne kolone (4 miliona podataka). 

In [6]:
df1 = pd.read_csv('../data/crimes.csv', usecols=['DATE OCC','AREA NAME','Crm Cd Desc','LOCATION'])
df1.sample(8)

,DATE OCC,AREA NAME,Crm Cd Desc,LOCATION
166403,01/24/2020 12:00:00 AM,Topanga,"BUNCO, PETTY THEFT",7700 ALABAMA AV
999949,01/22/2024 12:00:00 AM,Van Nuys,TRESPASSING,14100 CANTLAY ST
556704,08/11/2022 12:00:00 AM,Northeast,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),4200 EAGLE ROCK BL
820529,09/03/2023 12:00:00 AM,Southwest,INTIMATE PARTNER - AGGRAVATED ASSAULT,2600 HILLCREST DR
778282,02/19/2023 12:00:00 AM,Central,SHOPLIFTING - PETTY THEFT ($950 & UNDER),1000 S BROADWAY
107628,06/21/2020 12:00:00 AM,Rampart,INTIMATE PARTNER - AGGRAVATED ASSAULT,11TH ST
819149,08/08/2023 12:00:00 AM,Northeast,THEFT PLAIN - PETTY ($950 & UNDER),4600 FINLEY AV
884158,02/20/2024 12:00:00 AM,Olympic,BURGLARY,4000 W 3RD ST


#### Chunk-based Processing (Obrada u segmentima): učitavanje i obrada podataka u manjim delovima (chunks) pomoću parametra chunksize

In [31]:
for chunk in pd.read_csv('../data/crimes.csv',chunksize=10**5):
    print(chunk.shape)

(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(100000, 28)
(3448, 28)


## Sparse strukture podataka
#### Sparse (retki) podaci u većem obimu sadrže vrednosti koje ne nose nikakvu informaciju (nule, None vrednosti i sl.). Primer takve strukture bila bi sl.lista: 
#### [1, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0]
#### Sparse strukture podataka u Pandasu koriste se za efikasno skladištenje podataka koji imaju mnogo ponavljajućih vrednosti, obično nule ili NaN. Sparse objekti smanjuju memorijsko zauzeće tako što čuvaju samo značajne vrednosti i njihove indekse.
#### Pandas podržava sparse strukture kroz module poput pandas.SparseArray i pandas.SparseDataFrame.

In [1]:
data = [0, 0, 1, 0, 0, 2, 0, 0, 0, 3]
series = pd.Series(data)
series

0    0
1    0
2    1
3    0
4    0
5    2
6    0
7    0
8    0
9    3
dtype: int64

#### Nakon što smo od liste napravili pandas seriju, konvertovaćemo je u sparse seriju, uz pomoć SparseArray modula.

In [7]:
sparse_series = pd.Series(pd.arrays.SparseArray(data))
sparse_series

0    0
1    0
2    1
3    0
4    0
5    2
6    0
7    0
8    0
9    3
dtype: Sparse[int64, 0]

#### Naizgled deluje kao da smo dobili identičnu seriju, međutim, razlike u memorijskom zauzeću postaju izrazito primetne kod serija većih dimenzija. Kreiraćemo listu sa milion nula i tri nenulta elementa na kraju liste.

In [8]:
large_data = [0] * 1000000 + [1, 2, 3]

dense_series = pd.Series(large_data)

sparse_series_large = pd.Series(pd.arrays.SparseArray(large_data))

print("\nMemorijsko zauzeće (gusta serija):", dense_series.memory_usage(deep=True))
print("Memorijsko zauzeće (sparse serija):", sparse_series_large.memory_usage(deep=True))


Memorijsko zauzeće (gusta serija): 8000156
Memorijsko zauzeće (sparse serija): 168


#### Na osnovu poređenja memorijskog zauzeća obične i sparse serije, primećujemo da je ušteda memorijskog prostora skoro 8 miliona bajtova (8 megabajta). Sem uštede memorije, prednosti sparse struktura je kompatibilnost sa Pandas funkcijama: (sparse objekti podržavaju većinu standardnih Pandas operacija), kao i efikasnije računanje (operacije na sparse objektima mogu biti brže zbog smanjenja veličine podataka).

In [12]:
data = {
    'A': [0, 0, 1, 0],
    'B': [0, 2, 0, 0],
    'C': [0, 0, 0, 3]
}
df = pd.DataFrame(data)

sparse_df = df.astype(pd.SparseDtype("float", 0))

print(df)
print(sparse_df)

   A  B  C
0  0  0  0
1  0  2  0
2  1  0  0
3  0  0  3
     A    B    C
0    0    0    0
1    0  2.0    0
2  1.0    0    0
3    0    0  3.0


#### Prikazali smo kreiranje sparse datafrejma na osnovu običnog. Prilikom ispisa takođe deluju identično, ali smo prikazali prethodno u čemu se ogledaju razlike. Takođe, kao što smo već naveli, sparse strukture podržavaju iste funkcije kao i regularne pandas serije i datafrejmovi.

In [15]:
sparse_df['D']=sparse_df['B']*3.47
sparse_df.groupby('A')['D'].sum()

A
0      6.94
1.0     0.0
Name: D, dtype: Sparse[float64, np.float64(0.0)]